# Load libraries

In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

/home/eduardo/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load dataset

In [2]:
# Skip date information
df1 = pd.read_csv('/home/eduardo/projects/recommender-data/combined_data_1.txt',
                 header=None, names=['Cust_Id', 'Rating'], usecols=[0,1])

In [4]:
# First row is the movie ID, following rows have customer id and rating given to movie 1
df1.head()

,Cust_Id,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [8]:
df1.dtypes

Cust_Id     object
Rating     float64
dtype: object

In [9]:
df1.shape

(24058263, 2)

In [17]:
df1.

AttributeError: 'DataFrame' object has no attribute 'summary'

In [ ]:
# Load remaining datasets
df2 = pd.read_csv('/home/eduardo/projects/recommender-data/combined_data_2.txt',
                 header=None, names=['Cust_Id', 'Rating'], usecols=[0,1])
df3 = pd.read_csv('/home/eduardo/projects/recommender-data/combined_data_3.txt',
                 header=None, names=['Cust_Id', 'Rating'], usecols=[0,1])
df4 = pd.read_csv('/home/eduardo/projects/recommender-data/combined_data_4.txt',
                 header=None, names=['Cust_Id', 'Rating'], usecols=[0,1])

print(df2.shape)
print(df3.shape)
print(df4.shape)